In [2]:
# Using dlib to detect the faces in an image

import cv2
import dlib
import os
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
from enum import Enum
from collections import OrderedDict

%matplotlib inline 

class Point(Enum):
    X = 0
    Y = 1

PATH = os.path.join("..","..","..","dependencies")
FILE_NAME = "shape_predictor_68_face_landmarks.dat"

IMAGE_PATH = os.path.join("..","..","images","face_ops")

predictor = dlib.shape_predictor(os.path.join(PATH,FILE_NAME))
detector = dlib.get_frontal_face_detector()


# define a dictionary that maps the indexes of the facial
# landmarks to specific face regions
FACIAL_LANDMARKS_IDXS = OrderedDict([
    ("mouth", (48, 68)),
    ("right_eyebrow", (17, 22)),
    ("left_eyebrow", (22, 27)),
    ("right_eye", (36, 42)),
    ("left_eye", (42, 48)),
    ("nose", (27, 36)),
    ("jaw", (0, 17))
])

def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((68, 2), dtype=dtype)

    # loop over the 68 facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, 68):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords




def get_points(image):
    # Note: this function will only recieve faces that have exactly one face
    
    dets = detector(image)
    for k, d in enumerate(dets):

            gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)     

            shape = shape_to_np(predictor(gray, d))

    return shape # where shape is a 68 points list of tuples 



In [3]:
import sys
import numpy as np
import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

%matplotlib inline

# Read points from text file
def readPoints(path) :
    # Create an array of points.
    points = [];
    
    # Read points
    with open(path) as file :
        for line in file :
            x, y = line.split()
            points.append((int(x), int(y)))
    

    return points

# Apply affine transform calculated using srcTri and dstTri to src and
# output an image of size.
def applyAffineTransform(src, srcTri, dstTri, size) :
    
    # Given a pair of triangles, find the affine transform.
    warpMat = cv2.getAffineTransform( np.float32(srcTri), np.float32(dstTri) )
    
    # Apply the Affine Transform just found to the src image
    dst = cv2.warpAffine( src, warpMat, (size[0], size[1]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101 )

    return dst


# Check if a point is inside a rectangle
def rectContains(rect, point) :
    if point[0] < rect[0] :
        return False
    elif point[1] < rect[1] :
        return False
    elif point[0] > rect[0] + rect[2] :
        return False
    elif point[1] > rect[1] + rect[3] :
        return False
    return True


#calculate delanauy triangle
def calculateDelaunayTriangles(rect, points):
    #create subdiv
    subdiv = cv2.Subdiv2D(rect);
    
    # Insert points into subdiv
    for p in points:
        subdiv.insert(p) 
    
    triangleList = subdiv.getTriangleList();
    
    delaunayTri = []
    
    pt = []    
        
    for t in triangleList:        
        pt.append((t[0], t[1]))
        pt.append((t[2], t[3]))
        pt.append((t[4], t[5]))
        
        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])        
        
        if rectContains(rect, pt1) and rectContains(rect, pt2) and rectContains(rect, pt3):
            ind = []
            #Get face-points (from 68 face detector) by coordinates
            for j in xrange(0, 3):
                for k in xrange(0, len(points)):                    
                    if(abs(pt[j][0] - points[k][0]) < 1.0 and abs(pt[j][1] - points[k][1]) < 1.0):
                        ind.append(k)    
            # Three points form a triangle. Triangle array corresponds to the file tri.txt in FaceMorph 
            if len(ind) == 3:                                                
                delaunayTri.append((ind[0], ind[1], ind[2]))
        
        pt = []        
            
    
    return delaunayTri
        

# Warps and alpha blends triangular regions from img1 and img2 to img
def warpTriangle(img1, img2, t1, t2) :

    # Find bounding rectangle for each triangle
    r1 = cv2.boundingRect(np.float32([t1]))
    r2 = cv2.boundingRect(np.float32([t2]))

    # Offset points by left top corner of the respective rectangles
    t1Rect = [] 
    t2Rect = []
    t2RectInt = []

    for i in xrange(0, 3):
        t1Rect.append(((t1[i][0] - r1[0]),(t1[i][1] - r1[1])))
        t2Rect.append(((t2[i][0] - r2[0]),(t2[i][1] - r2[1])))
        t2RectInt.append(((t2[i][0] - r2[0]),(t2[i][1] - r2[1])))


    # Get mask by filling triangle
    mask = np.zeros((r2[3], r2[2], 3), dtype = np.float32)
    cv2.fillConvexPoly(mask, np.int32(t2RectInt), (1.0, 1.0, 1.0), 16, 0);

    # Apply warpImage to small rectangular patches
    img1Rect = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]
    #img2Rect = np.zeros((r2[3], r2[2]), dtype = img1Rect.dtype)
    
    size = (r2[2], r2[3])

    img2Rect = applyAffineTransform(img1Rect, t1Rect, t2Rect, size)
    
    img2Rect = img2Rect * mask

    # Copy triangular region of the rectangular patch to the output image
    img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] = img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] * ( (1.0, 1.0, 1.0) - mask )
     
    img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] = img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] + img2Rect 
    


    
img1 = mpimg.imread(os.path.join(IMAGE_PATH,"ang.jpg"))
img2 = mpimg.imread(os.path.join(IMAGE_PATH,"kate.jpg"))

img1Warped = np.copy(img2);    
    
# Read array of corresponding points
points1 = get_points(img1)
points2 = get_points(img2)
    
# Find convex hull
hull1 = []
hull2 = []

hullIndex = cv2.convexHull(np.array(points2), returnPoints = False)
          
for i in range(0, len(hullIndex)):
    hull1.append(points1[int(hullIndex[i])])
    hull2.append(points2[int(hullIndex[i])])
    
"""    
# Find delanauy traingulation for convex hull points
sizeImg2 = img2.shape    
rect = (0, 0, sizeImg2[1], sizeImg2[0])
     
dt = calculateDelaunayTriangles(rect, hull2)
    
if len(dt) == 0:
    quit()
    
# Apply affine transformation to Delaunay triangles
for i in range(0, len(dt)):
    t1 = []
    t2 = []
        
    #get points for img1, img2 corresponding to the triangles
    for j in xrange(0, 3):
        t1.append(hull1[dt[i][j]])
        t2.append(hull2[dt[i][j]])
        
    warpTriangle(img1, img1Warped, t1, t2)
    
            
# Calculate Mask
hull8U = []
for i in range(0, len(hull2)):
    hull8U.append((hull2[i][0], hull2[i][1]))
    
mask = np.zeros(img2.shape, dtype = img2.dtype)  
    
cv2.fillConvexPoly(mask, np.int32(hull8U), (255, 255, 255))
    
r = cv2.boundingRect(np.float32([hull2]))    
    
center = ((r[0]+int(r[2]/2), r[1]+int(r[3]/2)))
        
    
# Clone seamlessly.
output = cv2.seamlessClone(np.uint8(img1Warped), img2, mask, center, cv2.NORMAL_CLONE)
    
#plt.imshow("Face Swapped", output)
#plt.show()

"""

'    \n# Find delanauy traingulation for convex hull points\nsizeImg2 = img2.shape    \nrect = (0, 0, sizeImg2[1], sizeImg2[0])\n     \ndt = calculateDelaunayTriangles(rect, hull2)\n    \nif len(dt) == 0:\n    quit()\n    \n# Apply affine transformation to Delaunay triangles\nfor i in range(0, len(dt)):\n    t1 = []\n    t2 = []\n        \n    #get points for img1, img2 corresponding to the triangles\n    for j in xrange(0, 3):\n        t1.append(hull1[dt[i][j]])\n        t2.append(hull2[dt[i][j]])\n        \n    warpTriangle(img1, img1Warped, t1, t2)\n    \n            \n# Calculate Mask\nhull8U = []\nfor i in range(0, len(hull2)):\n    hull8U.append((hull2[i][0], hull2[i][1]))\n    \nmask = np.zeros(img2.shape, dtype = img2.dtype)  \n    \ncv2.fillConvexPoly(mask, np.int32(hull8U), (255, 255, 255))\n    \nr = cv2.boundingRect(np.float32([hull2]))    \n    \ncenter = ((r[0]+int(r[2]/2), r[1]+int(r[3]/2)))\n        \n    \n# Clone seamlessly.\noutput = cv2.seamlessClone(np.uint8(img1Wa